In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
papers = pd.read_csv('/kaggle/input/nips-papers-1987-2019-updated/papers.csv')
authors = pd.read_csv('/kaggle/input/nips-papers-1987-2019-updated/authors.csv')

In [ ]:
papers.head()

In [ ]:
authors.head()

### This time we don't have to use 'authors' data
#### so, let's seek the papers' data

#### Preproceccing

In [ ]:
papers.info()

#### we can't fill the abstract columns with some representative value because of objct column
#### and full_text too

#### But I think we don't have to fill abs col with some values because we don't use this col this time.
#### so It's enough to cut nulls of full_text col.

In [ ]:
papers = papers[~papers['full_text'].isnull()]
papers.info()

#### OK, let's move

#### First, we check some values in full_text to know what and how we should preprocess.

In [ ]:
papers['full_text'][0][:1000]

In [ ]:
papers['full_text'][1000][:1000]

#### I see.
#### I think we prepare morphs and clean up stopwords and regular expressions.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords


nltk.download("stopwords")

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
def cleaning_words(text):
    """
    cleaning the sentence 
    """
    
    text = text.lower()
    
    # Number doen't look necessary this time 
    text = re.sub('\d', '', text)
    text = re.sub('([\-])|\\n', '', text)
    
    text = text.split(' ')
    
    # cut the \n or \t
    strip_list = [word.strip() for word in text if not word in stop_words]
    
    strip_list = ' '.join(strip_list)
    
    return strip_list

In [ ]:
# apply the above function
papers['full_text_cleaned'] = papers['full_text'].apply(cleaning_words)
papers['full_text_cleaned']

#### OK, We try countvec and LDA action for the moment

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
corpus = papers['full_text_cleaned'].values
count = CountVectorizer(stop_words='english', max_df=.5, min_df=3)
X = count.fit_transform(corpus)

In [ ]:
lda = LatentDirichletAllocation(n_components=15, random_state=123)
X_topics = lda.fit_transform(X)

In [ ]:
topn = 5
feature_names = count.get_feature_names()

for topix_idx, topic in enumerate(lda.components_):
    print(f'Topic: {topix_idx + 1}')
    indices = topic.argsort()[:-topn - 1: -1]
    
    words = ' '.join([feature_names[i] for i in indices])
    print(words)
    print('-'*10)

### We can measure Coherence and something..
### but this codes are a little complex so we just adjust 15 -> 10 topic with manually

In [ ]:
lda_10 = LatentDirichletAllocation(n_components=10, random_state=123)
X_topics = lda_10.fit_transform(X)

In [ ]:
topn = 5
feature_names = count.get_feature_names()

for topix_idx, topic in enumerate(lda_10.components_):
    print(f'Topic: {topix_idx + 1}')
    indices = topic.argsort()[:-topn - 1: -1]
    
    words = ' '.join([feature_names[i] for i in indices])
    print(words)
    print('-'*10)

### We get to know the topic meaning according to this topn words

#### topic1-> about loss function or something?
#### topic2-> about neurons 
#### topic3-> about gaussian kernel!
#### topic4-> about Bayesian inference for latent　valuable model?
#### topic5-> about image feature(-> CNN? clustering?? haha)
#### topic6-> about SARSA（state–action–reward–state–action）?
#### topic7-> about arxiv 
#### topic8-> about graph nodes (-> random forest? or somethin? haha)
#### topic9-> about Gradient theorem?
#### topic10-> about clustering and cost function?


I keep trying this data but I suspend for the moment 
See you soon again !